<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit3/nlp_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install transformers[sentencepiece] datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")
training_args

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    data_collator=data_collator, # optional since datacollator is provided in tokenizer
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,

)

In [8]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.624100
1000,0.428400


TrainOutput(global_step=1377, training_loss=0.46579980677076244, metrics={'train_runtime': 195.6258, 'train_samples_per_second': 56.25, 'train_steps_per_second': 7.039, 'total_flos': 405626802939840.0, 'train_loss': 0.46579980677076244, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(tokenized_datasets['validation'])
predictions

In [32]:
predictions.predictions

array([[-2.7929053 ,  2.6267154 ],
       [ 1.874778  , -1.8134218 ],
       [-0.2273959 , -0.2952312 ],
       [-2.4451299 ,  2.272764  ],
       [ 1.8080478 , -1.7389238 ],
       [-2.7542586 ,  2.6042478 ],
       [-2.683237  ,  2.5493689 ],
       [-2.774623  ,  2.6176164 ],
       [-2.4860506 ,  2.3262224 ],
       [-2.739922  ,  2.5962155 ],
       [-2.7900567 ,  2.6261508 ],
       [ 1.7799718 , -1.7159178 ],
       [ 2.1334429 , -2.0764174 ],
       [-2.6529813 ,  2.5213804 ],
       [-2.7833622 ,  2.622381  ],
       [-2.496613  ,  2.339938  ],
       [-2.7918    ,  2.6261578 ],
       [ 1.6867297 , -1.7307473 ],
       [-2.795071  ,  2.627453  ],
       [ 1.7095538 , -1.7188561 ],
       [ 1.7694947 , -1.7521127 ],
       [-1.4872586 ,  1.0038283 ],
       [ 1.7581935 , -1.7377706 ],
       [-2.7679138 ,  2.6143534 ],
       [-2.786296  ,  2.6240053 ],
       [ 0.8071619 , -1.1157502 ],
       [-1.9387144 ,  1.6920979 ],
       [-2.7978134 ,  2.628954  ],
       [-2.654278  ,

In [36]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=1)
preds

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [29]:
import evaluate

metrics = evaluate.load("glue", "mrpc")
metrics.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8161764705882353, 'f1': 0.8691099476439791}

In [46]:
def compute_metrics(eval_preds):
  metrics = evaluate.load("glue", "mrpc")
  logits, labels = eval_preds
  preds = np.argmax(logits, axis=-1)
  returned_metrics = metrics.compute(predictions=preds,references=labels)
  returned_metrics["drewno"] = 3
  return returned_metrics



In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

training_args = TrainingArguments("test-trainer2", evaluation_strategy="epoch")

trainer = Trainer(
    model,
    training_args,
    data_collator=data_collator, # optional since datacollator is provided in tokenizer
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()